In [ ]:
import numpy as np
from open3d import *  
import torch
import seaborn as sns
import pandas as pd
import time
import os

In [ ]:
from eval_det import get_iou, voc_ap

In [ ]:
fig_path = 'C:\\Users\\Lorenz\\Documents\\hs-vcml\\book\\figures\\chapter_1'

In [ ]:
s3dis_label_dict = {
    0: 'ceiling',
    1: 'floor',
    2: 'wall',
    3: 'beam',
    4: 'column',
    5: 'window',
    6: 'door',
    7: 'chair',
    8: 'table',
    9: 'bookcase',
    10: 'sofa',
    11: 'board',
    12: 'clutter',
}

CLASSES = ('floor', 'building', 'low vegetation', 'med. vegetation', 'high vegetation', 'vehicle',
               'truck', 'aircraft', 'militaryVehicle', 'bike', 'motorcycle', 'light pole',
               'street sign', 'clutter', 'fence')
stpls3d_label_dict = {i: name for i, name in enumerate(CLASSES)}

In [ ]:
def calc_color_vec(labels):
    index_label_dict = {int(value):int(index[0]) for index, value in np.ndenumerate(np.unique(labels))}
    color_codes = list(sns.color_palette("husl", len(np.unique(labels))))
    colors = [color_codes[index_label_dict[int(label)]] for label in np.nditer(labels)]
    return colors

In [ ]:
def rgb_to_hex(rgb):
    rgb = tuple(int(v*255) for v in rgb)
    return '#%02x%02x%02x' % rgb

def create_colors_from_label_dict(label_vec, label_dict):
    labels = list(label_dict.keys())
    color_codes = list(sns.color_palette("Spectral", len(np.unique(labels))))
    hex_list = [rgb_to_hex(c) for c in color_codes]
    name_color_dict = {label_dict[label]: color for label, color in zip(labels, hex_list)}

    df = pd.DataFrame(hex_list).T

    def color_positive_green(col):
        return [f'background-color: {name_color_dict[col.name]}']

    df.iloc[0,0:len(hex_list)] = ""
    df.columns = list(label_dict.values()) 
    display(df.style.apply(color_positive_green))
    return np.array([color_codes[label] for label in np.nditer(label_vec)])

In [ ]:
torr_view_json = {
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : False,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 1.8517245054244995, 501.92291259765625, 157.94999694824219 ],
			"boundingbox_min" : [ -138.82221984863281, 361.89385986328125, 131.83200073242188 ],
			"field_of_view" : 60.0,
			"front" : [ 0.89995929372347894, -0.41732420993349789, 0.12614980556512517 ],
			"lookat" : [ -68.485247671604156, 431.90838623046875, 144.89099884033203 ],
			"up" : [ -0.10413216502600244, 0.075217200951982532, 0.9917151127657311 ],
			"zoom" : 0.2999999999999996
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

In [ ]:
view_json = {
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : False,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 29.079366683959961, 34.424301147460938, 12.430385589599609 ],
			"boundingbox_min" : [ -37.939289093017578, -32.362068176269531, -4.4400639533996582 ],
			"field_of_view" : 60.0,
			"front" : [ -0.94616145287019626, 0.18772444699202229, 0.26370065814119409 ],
			"lookat" : [ -4.4299612045288086, 1.0311164855957031, 3.9951608180999756 ],
			"up" : [ 0.26480866917737744, -0.019616132576752879, 0.96410143453437391 ],
			"zoom" : 0.5199999999999998
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}


def create_image(pcd, output_path, view_json):
    if os.path.isfile(output_path):
        print(f'Warning, file "{output_path}" already exists!')
        return
    vis = open3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)

    view_control = vis.get_view_control()
    view_control.set_front(view_json['trajectory'][0]['front'])
    view_control.set_zoom(view_json['trajectory'][0]['zoom'])
    view_control.set_lookat(view_json['trajectory'][0]['lookat'])
    view_control.set_up(view_json['trajectory'][0]['up'])

    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()

    time.sleep(1)
    vis.capture_screen_image(output_path)
    vis.remove_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()
    vis.destroy_window()
    del view_control
    del vis

In [ ]:
# torronto dataset
path = 'data/results_new'
file_name = '25_points_GTv3_098.npy'

In [ ]:
# original dataset
path = 'data/results_manipulated/sample_farthest-0.5/results_stpls3d'
file_name = '5_points_GTv3_05.npy'

## Scence with color

In [ ]:
cords = np.load(f'{path}/coords/{file_name}')
colors = np.load(f'{path}/colors/{file_name}')
colors = (colors + 1) / 2

In [ ]:
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(cords[mask])
pcd.colors = open3d.utility.Vector3dVector(colors[mask])

In [ ]:
open3d.visualization.draw_geometries([pcd])

In [ ]:
create_image(pcd, f'{fig_path}\\toronto3d-color.png', torr_view_json)

## Scence with label

In [ ]:
cords = np.load(f'{path}/coords/{file_name}')
labels = np.load(f'{path}/semantic_label/{file_name}')

In [ ]:
colors = create_colors_from_label_dict(labels, stpls3d_label_dict)
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(cords[mask])
pcd.colors = open3d.utility.Vector3dVector(colors[mask])

In [ ]:
open3d.visualization.draw_geometries([pcd])

In [ ]:
#create_image(pcd, f'{fig_path}\\manipulated-data-noisy-labels.png', view_json)

## Scence with prediction

In [ ]:
cords = np.load(f'{path}/coords/{file_name}')
predictions = np.load(f'{path}/semantic_pred/{file_name}')

In [ ]:
colors = create_colors_from_label_dict(predictions, stpls3d_label_dict)
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(cords[mask])
pcd.colors = open3d.utility.Vector3dVector(colors[mask])

In [ ]:
open3d.visualization.draw_geometries([pcd])

In [ ]:
#create_image(pcd, f'{fig_path}\\toronto3d-prediction-labels.png', torr_view_json)

## Accuracy Farbencodiert

In [ ]:
cords = np.load(f'{path}/coords/{file_name}')
colors = np.load(f'{path}/colors/{file_name}')
colors = (colors + 1) / 2

In [ ]:
acc = labels == predictions
acc_colors = np.array([np.logical_not(acc)*255, acc*255, acc*0]).T
incorrect = np.logical_not(acc)
colors[incorrect] = acc_colors[incorrect]
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(cords[mask])
pcd.colors = open3d.utility.Vector3dVector(colors[mask])

In [ ]:
open3d.visualization.draw_geometries([pcd])

In [ ]:
create_image(pcd, f'{fig_path}\\manipulated-data-noisy-errors-colorcoded.png', view_json)